In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
class CustomLayer(nn.Module):
    def __init__(self, in_features: int, out_features: int, bias: bool = True, device=None, dtype=None):
        factory_kwargs = {'device': device, 'dtype': dtype}
        super(CustomLayer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.empty((out_features, in_features), **factory_kwargs))
        if bias:
            self.bias = nn.Parameter(torch.empty(out_features, **factory_kwargs))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def forward(self, s_k, s_l_minus_1, P_l_minus_1):
        exp_term = torch.exp(-self.B(s_k))
        product = 1.0

        for i in range(self.n):
            integrand = P_l_minus_1[i] * s_l_minus_1[i] * torch.exp(
                -self.w[i] * ((s_k - s_l_minus_1[i]) / self.q)**2
            )
            product *= integrand

        result = exp_term * product
        return result


'1.13.1+cu117'

In [29]:
a = torch.Tensor(torch.ones((2,5)))
a.size

tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])

In [16]:
in_features, out_features = 10, 20
nn.Parameter(torch.empty((out_features, in_features)))

Parameter containing:
tensor([[        nan, -2.2070e+38,  1.8431e-18,  8.2939e-18,  1.5516e-36,
          4.8721e-31,  5.5599e-38, -3.8221e+02,  1.6615e+20,  4.5660e-41],
        [ 6.0384e+18,  4.5660e-41,  1.8347e+07, -9.1016e+20,  1.6615e+20,
          4.5660e-41,  6.0383e+18,  4.5660e-41,  7.2017e+06,  3.9201e-09],
        [ 1.6615e+20,  4.5660e-41,  2.5385e+19,  4.5660e-41,  2.0383e+23,
         -8.7250e+07,  8.1462e+21,  4.5660e-41,  6.0384e+18,  4.5660e-41],
        [ 1.3544e-07, -2.3314e+36,  1.6615e+20,  4.5660e-41,  6.0383e+18,
          4.5660e-41, -3.4410e-19,  2.1494e-35,  2.4894e+20,  4.5660e-41],
        [ 6.0098e+18,  4.5660e-41,  8.4344e+22, -3.2836e-18,  1.0276e+22,
          4.5660e-41,  6.0055e+18,  4.5660e-41, -1.6114e-15, -1.1619e-36],
        [ 9.3966e+21,  4.5660e-41,  6.0055e+18,  4.5660e-41,  1.1916e+34,
         -1.5616e-08,  1.0546e+22,  4.5660e-41,  1.0325e+22,  4.5660e-41],
        [ 2.0871e+14,  2.3921e+25,  1.0544e+22,  4.5660e-41,  1.6448e+20,
          

In [ ]:
class YourNetwork(nn.Module):
    def __init__(self, ...):
        super(YourNetwork, self).__init__()
        # Other layers

        # Initialize the custom layer
        self.custom_layer = CustomLayer(n, q, B, w)

    def forward(self, x):
        # Other forward propagation steps

        # Use the custom layer
        custom_output = self.custom_layer(s_k, s_l_minus_1, P_l_minus_1)

        # Other forward propagation steps


In [24]:
import torch
import torch.nn as nn

class CustomLinear(nn.Module):
    def __init__(self, in_features, out_features):
        super(CustomLinear, self).__init__()
        self.weight = nn.Parameter(torch.randn(out_features, in_features))
        self.bias = nn.Parameter(torch.randn(out_features))

    def forward(self, x):
        return torch.matmul(x, self.weight.t()) + self.bias

# Create an instance of the CustomLinear layer
linear_layer = CustomLinear(in_features=3, out_features=9)

# Create a sample input tensor
input_data = torch.randn(2, 3)  # Batch size of 2, input dimension of 3

# Pass the input through the linear layer
output = linear_layer(input_data)
print("Output shape:", output.shape)
print(output)

Output shape: torch.Size([2, 9])
tensor([[-1.2061, -1.6754, -0.7176,  0.6579, -1.1079, -0.1029,  0.1930,  1.4293,
          0.5787],
        [ 1.6930,  1.2094, -0.4598, -3.2865, -2.1971, -3.0835,  3.0158,  1.7780,
          4.5608]], grad_fn=<AddBackward0>)


In [25]:
print(input_data)

tensor([[-0.3692,  0.5129, -0.1973],
        [-0.0369, -0.7358,  2.0560]])
